# Load Package

In [1]:
# !pip install -U scikit-learn
# !pip install surprise
# !pip install networkx
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
import scipy
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tqdm import tqdm
import warnings
# import networkx as nx
# warnings.filterwarnings('ignore')
# from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
# from surprise.model_selection import cross_validate
# from surprise import NormalPredictor
# from surprise import Dataset
# from surprise import Reader

# Load Data

In [2]:
ratings_Digital_Music= pd.read_csv('ratings_Digital_Music.csv', names=["user","item","rating","timestamp"])

In [3]:
ratings_Digital_Music.head()

,user,item,rating,timestamp
0,A2EFCYXHNK06IS,5555991584,5.0,978480000
1,A1WR23ER5HMAA9,5555991584,5.0,953424000
2,A2IR4Q0GPAFJKW,5555991584,4.0,1393545600
3,A2V0KUVAB9HSYO,5555991584,4.0,966124800
4,A1J0GL9HCA7ELW,5555991584,5.0,1007683200


In [4]:
print("There are {} users, {} ratings, {} items.".format(len(ratings_Digital_Music["user"].unique()), ratings_Digital_Music.shape[0], len(ratings_Digital_Music["item"].unique())))

There are 478235 users, 836006 ratings, 266414 items.


In [5]:
# user_count = pd.DataFrame(ratings_Digital_Music.value_counts(subset=["user"]), columns=["count"])
# user_count.reset_index(drop=False, inplace=True)
# user_count.head()
user_count = pd.DataFrame(ratings_Digital_Music["user"].value_counts())
user_count.reset_index(drop=False, inplace=True)
user_count.columns = ['user','count']
user_count.head()

,user,count
0,A3W4D8XOGLWUN5,1126
1,A9Q28YTLYREO7,713
2,ABDR6IJ93HFIO,489
3,A3HU0B9XUEVHIM,471
4,A1GN8UJIZLCA59,427


In [6]:
valid_user = user_count[user_count["count"] >= 10].user.tolist()

# Simplify Data

In [7]:
rating_sample = ratings_Digital_Music.sample(frac=0.025, replace=False, random_state=1)
rating_sample.reset_index(drop=True, inplace=True)
print("There are {} users, {} ratings, {} items.".format(len(rating_sample["user"].unique()), rating_sample.shape[0], len(rating_sample["item"].unique())))

There are 19524 users, 20900 ratings, 14946 items.


In [8]:
rating_valid = ratings_Digital_Music[ratings_Digital_Music["user"].isin(valid_user)]

In [9]:
rating_valid.reset_index(drop=True, inplace=True)
rating_valid.head()

,user,item,rating,timestamp
0,A3EBHHCZO6V2A4,5555991584,5.0,1158019200
1,AZPWAXJG9OJXV,5555991584,5.0,991526400
2,A38IRL0X2T4DPF,5555991584,5.0,1058140800
3,A2P49WD75WHAG5,5555991584,5.0,1059523200
4,A3O90G1D7I5EGG,5555991584,3.0,959385600


# Using Surprise Package

In [ ]:
for selected_data in [rating_sample, rating_valid]:
  reader = Reader(rating_scale=(1, 5))
  data = Dataset.load_from_df(selected_data[["user","item","rating"]], reader)
  print(cross_validate(NormalPredictor(), data, cv=2))

  dataset = data.build_full_trainset()
  from surprise.model_selection import train_test_split
  X_train, X_test = train_test_split(data, test_size=0.25)

  selected_algorithm = [SVDpp(), KNNBaseline(), KNNWithZScore()]
  algorithm_name = ['SVDpp()', 'KNNBaseline()', 'KNNWithZScore()']

  from surprise import accuracy
  reader = Reader(rating_scale=(1, 5))
  trainset = X_train
  testset = X_test
  for i in range(len(selected_algorithm)):
    algo = selected_algorithm[i]
    name = algorithm_name[i]
    predictions = algo.fit(trainset).test(testset)
    print('{} has MAE {}'.format(name, accuracy.mae(predictions)))

{'test_rmse': array([1.21184112, 1.20051926]), 'test_mae': array([0.84248393, 0.82229975]), 'fit_time': (0.016071557998657227, 0.019555091857910156), 'test_time': (0.09179949760437012, 0.3361515998840332)}
MAE:  0.6790
SVDpp() has MAE 0.6789973377071562
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.6824
KNNBaseline() has MAE 0.6823752015753572
Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.6866
KNNWithZScore() has MAE 0.6866204848063322
{'test_rmse': array([1.17699171, 1.18836502]), 'test_mae': array([0.84946855, 0.85947937]), 'fit_time': (0.11156797409057617, 0.1468977928161621), 'test_time': (0.5313498973846436, 0.7437963485717773)}
MAE:  0.5249
SVDpp() has MAE 0.5248613731088184
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.5772
KNNBaseline() has MAE 0.5771630655719845
Computing the msd similarity matrix...
Done computi

# Using SVD

In [ ]:
def compute_svd(urm, K):
    U, s, Vt = svds(urm, K)
 
    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])
 
    U = csc_matrix(U, dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

def compute_estimated_matrix(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 
    max_recommendation = 250
    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    recomendRatings = np.zeros(shape=(MAX_UID,max_recommendation ), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm
        estimatedRatings[userTest, :] = prod.todense()
        recomendRatings[userTest, :] = (-estimatedRatings[userTest, :]).argsort()[:max_recommendation]
    return estimatedRatings, recomendRatings


In [ ]:
from scipy.sparse import coo_matrix
import math as mt
from scipy.sparse.linalg import * #used for matrix multiplication
from scipy.sparse.linalg import svds
from scipy.sparse import csc_matrix

df_set = [rating_sample, rating_valid, ratings_Digital_Music]

for small_set in df_set:
  user_codes = small_set.user.drop_duplicates().reset_index()
  song_codes = small_set.item.drop_duplicates().reset_index()

  user_codes.rename(columns={'index':'user_index'}, inplace=True)
  song_codes.rename(columns={'index':'song_index'}, inplace=True)
  
  song_codes['so_index_value'] = list(song_codes.index)
  user_codes['us_index_value'] = list(user_codes.index)

  small_set = pd.merge(small_set,song_codes,how='left')
  small_set = pd.merge(small_set,user_codes,how='left')

  mat_candidate = small_set[['us_index_value','so_index_value','rating']]

  df = mat_candidate
  train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
  data_array = train.rating.values
  row_array = train.us_index_value.values
  col_array = train.so_index_value.values

  data_sparse = coo_matrix((data_array, (row_array, col_array)),dtype=float)

  K=10
  urm = data_sparse
  MAX_PID = urm.shape[1]
  MAX_UID = urm.shape[0]

  U, S, Vt = compute_svd(urm, K)
  uTest = test.us_index_value.tolist()
  uTest_estimated_ratings, uTest_recommended_items = compute_estimated_matrix(urm, U, S, Vt, uTest, K, True)

  test_data_array = test.rating.values
  test_row_array = test.us_index_value.values
  test_col_array = test.so_index_value.values

  mae = 0
  for i in range(len(test_row_array)):
    r = test_row_array[i]
    c = test_col_array[i]
    try:
      mae += abs(uTest_estimated_ratings[r, c]-test_data_array[i])
    except:
      continue
  mae /= len(test_row_array)
  print(mae)

4.511244019138756
4.449877449120132


# Using Sklearn

In [10]:
rating_sample_copy = rating_sample.copy()
rating_valid_copy = rating_valid.copy()

In [11]:
for dataset in [rating_sample_copy, rating_valid_copy]:
  for col in ["user", "item"]:
    selCol = dataset[col]
    for i in range(len(selCol)):
      temp = ''
      for c in selCol[i]:
        if c.isdigit():
          temp += c
        else:
          temp += str(ord(c))
      selCol[i] = temp
  # for i in range(len(item)):
  #   temp = ''
  #   for c in item[i]:
  #     if c.isdigit():
  #       temp += c
  #     else:
  #       temp += str(ord(c))
  #   item[i] = temp

<ipython-input-11-e480f6e6f152>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selCol[i] = temp


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
for selected_data in [rating_sample_copy, rating_valid_copy]:
  x_trainML = selected_data[["user","item"]]
  y_trainML = selected_data["rating"]
  X_train, X_test, y_train, y_test =sklearn.model_selection.train_test_split(x_trainML, y_trainML, test_size=0.25)

  selected_algorithm = [LinearSVC(), RandomForestClassifier(), MLPClassifier()]
  algorithm_name = ["LinearSVC()", "RandomForestClassifier()", "MLPClassifier()"]

  # from surprise import accuracy
  # reader = Reader(rating_scale=(1, 5))
  # trainset = X_train
  # testset = X_test
  for i in range(len(selected_algorithm)):
    algo = selected_algorithm[i]
    name = algorithm_name[i]
    algo.fit(X_train, y_train)
    predictions = algo.predict(X_test)
    print('{} has mae {}'.format(name, sklearn.metrics.mean_absolute_error(y_test, predictions, multioutput='uniform_average')))

C:\Users\Qminjia\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC() has mae 2.4088038277511963
RandomForestClassifier() has mae 0.5431578947368421
MLPClassifier() has mae 1.731866028708134


C:\Users\Qminjia\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC() has mae 0.538958180484226
RandomForestClassifier() has mae 0.5608217168011739
MLPClassifier() has mae 1.6174614820249449


# Using tfidf

In [1]:
import json

In [17]:
data = []
with open("Digital_Music_5.json",'r') as load_f:
    temp = load_f.readline()
    while temp:
        row = []
        temp = json.loads(temp)
        row.append(temp['asin'])
        row.append(temp['reviewText'])
        row.append(temp['overall'])
        data.append(row)
        temp = load_f.readline()
#     break

In [20]:
comments_df = pd.DataFrame(data=data,columns=['item', 'review', 'rating'])

In [25]:
x_trainML = comments_df["review"]
y_trainML = comments_df["rating"]
X_train, X_test, y_train, y_test =sklearn.model_selection.train_test_split(x_trainML, y_trainML, test_size=0.25)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer = "word",  preprocessor = None, encoding='utf-8', ngram_range=(1, 4), sublinear_tf = True)
documentsVectors = vectorizer.fit_transform(x_trainML)

In [30]:
from sklearn import svm
size1 = len(X_train)
clf = svm.LinearSVC(C = 2.0)
clf.fit(documentsVectors[0:size1],y_train[0:size1])
prediction = clf.predict(documentsVectors[size1:])

In [47]:
sum = 0
for truth, pred in zip(y_test, prediction):
    sum += abs(truth-pred)

In [49]:
print('The mae is {}'.format(sum/len(prediction)))

The mae is 0.7730110650924151
